<a href="https://colab.research.google.com/github/sneharreddy25/project_2025/blob/main/Project_Next_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project: Next Word Prediction

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

In [32]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    # Splitting the text into sentences using
    # delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [33]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [34]:
# ... (Define and compile the model as before)

# Upload the weights file
uploaded_weights = files.upload()

# Assuming the uploaded file is named 'model_weights_after_445_epochs.h5'
# If you uploaded with a different name, change the key below accordingly
weights_filename = 'model_weights_after_495_epochs.h5'

# Check if the weights file was successfully uploaded
if weights_filename in uploaded_weights:
    # Load the saved weights
    model.load_weights(weights_filename)
    print(f"Successfully loaded weights from {weights_filename}")
else:
    print(f"Error: {weights_filename} was not found after upload. Please ensure the file is correctly uploaded.")


# Now continue training from epoch 496 to 500
model.fit(X, y, epochs=500, initial_epoch=495, verbose=1)

Saving pizza.txt to pizza (1).txt
Error: model_weights_after_495_epochs.h5 was not found after upload. Please ensure the file is correctly uploaded.
Epoch 496/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.0265 - loss: 6.3720
Epoch 497/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.0536 - loss: 5.7632
Epoch 498/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.0546 - loss: 5.6781
Epoch 499/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.0506 - loss: 5.6553
Epoch 500/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0610 - loss: 5.6272


In [35]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Next predicted words: Pizza have different  and and and and and
